# Numerai Crypto Competitie Voorspellingsmodel met H2O Sparkling Water

Dit notebook implementeert een voorspellingsmodel voor de Numerai/Numerai Crypto competitie met behulp van H2O Sparkling Water, wat H2O integreert met Apache Spark voor gedistribueerde verwerking.

## Installatie van benodigde packages

Eerst moeten we Java, Spark en H2O Sparkling Water installeren. Dit kan enige tijd duren.

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount("/content/drive")

# Create directory for Numerai data and models
!mkdir -p "/content/drive/My Drive/Numerai_Crypto"

In [ ]:
# Check if GPU is available
!nvidia-smi

In [ ]:
# Installeer Java (vereist voor H2O en Spark)
!apt-get update -qq
!apt-get install -y default-jre > /dev/null
!java -version

# Installeer Spark en PySpark
!pip install -q pyspark==3.1.2

# Installeer H2O Sparkling Water
!pip install -q h2o-pysparkling-3.1

# Installeer andere benodigde packages
!pip install -q numerapi pandas h2o cloudpickle==2.2.1 pyarrow scikit-learn scipy==1.10.1 matplotlib

## Importeren van benodigde libraries

In [ ]:
from numerapi import NumerAPI
import pandas as pd
import json
import os
import numpy as np
import time
import matplotlib.pyplot as plt

# Spark imports
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

# H2O Sparkling Water imports
from pysparkling import H2OContext
from pysparkling.ml import H2OXGBoostEstimator

import h2o
import cloudpickle

## Initialiseren van Spark en H2O Sparkling Water

In [ ]:
# Initialiseer Spark sessie
spark = SparkSession.builder \
    .appName("NumeraiSparklingWater") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .getOrCreate()

# Initialiseer H2O Sparkling Water context
h2o_context = H2OContext.getOrCreate()

# Print Spark en H2O versie informatie
print(f"Spark version: {spark.version}")
print(f"H2O cluster version: {h2o_context.getH2OVersion()}")
print(f"Sparkling Water version: {h2o_context.getSparklingWaterVersion()}")

## Initialiseren van de Numerai API

In [ ]:
# Initialiseer de Numerai API client
# Voor het indienen van voorspellingen zijn API keys nodig
# napi = NumerAPI(public_id="UW_PUBLIC_ID", secret_key="UW_SECRET_KEY")
napi = NumerAPI()

## Data downloaden en laden

In [ ]:
# Gebruik een van de nieuwste dataversies
DATA_VERSION = "v5.0"

# Maak een data directory
!mkdir -p {DATA_VERSION}

# Download data
print("Downloading training data...")
napi.download_dataset(f"{DATA_VERSION}/train.parquet")
napi.download_dataset(f"{DATA_VERSION}/features.json")

# Laad feature metadata
feature_metadata = json.load(open(f"{DATA_VERSION}/features.json"))
print("Available feature sets:", list(feature_metadata["feature_sets"].keys()))
features = feature_metadata["feature_sets"]["small"]  # gebruik "small" voor sneller testen, "medium" of "all" voor betere prestaties

## Data laden met PySpark

In [ ]:
# Laad trainingsdata met Spark
print("Loading training data with Spark...")
train_spark = spark.read.parquet(f"{DATA_VERSION}/train.parquet")

# Selecteer alleen de benodigde kolommen
columns_to_select = ["era"] + features + ["target"]
train_spark = train_spark.select(*columns_to_select)

# Downsampling voor snelheid (optioneel)
print("Preparing data for training...")
# Haal unieke era's op en sample 25% (elke 4e era)
unique_eras = [row.era for row in train_spark.select("era").distinct().collect()]
sampled_eras = unique_eras[::4]
train_spark = train_spark.filter(col("era").isin(sampled_eras))

# Bekijk de data
print(f"Training data count: {train_spark.count()}")
print(f"Number of features: {len(features)}")
print(f"Number of eras: {len(sampled_eras)}")

# Toon schema
train_spark.printSchema()

## Data voorbereiden met PySpark

In [ ]:
# Bereid data voor met Spark ML Pipeline
print("Preparing feature vector with Spark...")

# Maak een feature vector van alle features
assembler = VectorAssembler(inputCols=features, outputCol="features")
train_spark = assembler.transform(train_spark)

# Toon een voorbeeld van de getransformeerde data
train_spark.select("era", "features", "target").show(5, truncate=True)

## Converteren van Spark DataFrame naar H2O Frame

In [ ]:
# Converteer Spark DataFrame naar H2O Frame
print("Converting Spark DataFrame to H2O Frame...")
train_h2o = h2o_context.asH2OFrame(train_spark)

# Bekijk H2O Frame info
train_h2o.describe()

## Model trainen met H2O XGBoost via Sparkling Water

In [ ]:
# Train model met H2O XGBoost via Sparkling Water
print("Training H2O XGBoost model via Sparkling Water...")
start_time = time.time()

# Configureer XGBoost model
from h2o.estimators.xgboost import H2OXGBoostEstimator

xgb_model = H2OXGBoostEstimator(
    ntrees=2000,
    max_depth=5,
    learn_rate=0.01,
    sample_rate=0.8,
    col_sample_rate=0.8,
    tree_method="auto",  # auto selecteert GPU indien beschikbaar
    booster="gbtree",
    seed=42
)

# Train het model
xgb_model.train(x=features, y="target", training_frame=train_h2o)

training_time = time.time() - start_time
print(f"Training completed in {training_time:.2f} seconds")

# Toon model informatie
print(xgb_model)

## Feature importance visualiseren

In [ ]:
# Feature importance visualiseren
feature_importance = xgb_model.varimp(use_pandas=True)
if feature_importance is not None:
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(feature_importance[:20])), feature_importance[:20]['relative_importance'])
    plt.yticks(range(len(feature_importance[:20])), feature_importance[:20]['variable'])
    plt.title('H2O XGBoost Feature Importance (top 20)')
    plt.xlabel('Relative Importance')
    plt.tight_layout()
    plt.show()

## Model opslaan als MOJO

In [ ]:
# Sla het model op als MOJO (Model Object, Optimized)
mojo_path = xgb_model.download_mojo(path="./", get_genmodel_jar=True)
print(f"Model saved as MOJO: {mojo_path}")

## Validatiedata laden en voorbereiden met PySpark

In [ ]:
# Download validatiedata voor testen
print("Downloading validation data for testing...")
napi.download_dataset(f"{DATA_VERSION}/validation.parquet")

# Laad validatiedata met Spark
print("Loading validation data with Spark...")
validation_spark = spark.read.parquet(f"{DATA_VERSION}/validation.parquet")

# Selecteer alleen de benodigde kolommen
columns_to_select = ["era", "data_type"] + features
validation_spark = validation_spark.select(*columns_to_select)

# Filter alleen validatie data
validation_spark = validation_spark.filter(col("data_type") == "validation")

# Neem een kleine subset voor geheugenefficiëntie
validation_spark = validation_spark.limit(1000)

# Maak een feature vector van alle features
validation_spark = assembler.transform(validation_spark)

# Converteer Spark DataFrame naar H2O Frame
validation_h2o = h2o_context.asH2OFrame(validation_spark)

## Voorspellingen maken met het model

In [ ]:
# Maak voorspellingen met het model
print("Making predictions...")
predictions_h2o = xgb_model.predict(validation_h2o)

# Converteer H2O Frame terug naar Spark DataFrame
predictions_spark = h2o_context.asSparkFrame(predictions_h2o)

# Toon voorspellingen
print("Sample predictions:")
predictions_spark.show(5)

## Voorspellingsfunctie definiëren

In [ ]:
# Definieer voorspellingsfunctie die werkt met H2O model
def predict(
    live_features: pd.DataFrame,
    live_benchmark_models: pd.DataFrame
) -> pd.DataFrame:
    # Converteer pandas DataFrame naar Spark DataFrame
    live_features_spark = spark.createDataFrame(live_features[features])
    
    # Maak een feature vector van alle features
    live_features_spark = assembler.transform(live_features_spark)
    
    # Converteer Spark DataFrame naar H2O Frame
    live_features_h2o = h2o_context.asH2OFrame(live_features_spark)
    
    # Maak voorspellingen met het H2O model
    preds = xgb_model.predict(live_features_h2o)
    
    # Converteer H2O voorspellingen terug naar pandas
    predictions = h2o.as_list(preds)["predict"].values
    
    # Maak submission DataFrame
    submission = pd.Series(predictions, index=live_features.index)
    return submission.to_frame("prediction")

## Voorspellingsfunctie testen

In [ ]:
# Converteer Spark DataFrame terug naar pandas voor testen
validation_pd = validation_spark.toPandas()

# Test voorspellingsfunctie
print("Testing prediction function...")
# Maak een lege DataFrame voor benchmark_models (niet gebruikt in onze voorspellingsfunctie)
empty_benchmark = pd.DataFrame(index=validation_pd.index)
predictions = predict(validation_pd, empty_benchmark)

print(f"Predictions shape: {predictions.shape}")
print("\nSample predictions:")
print(predictions.head())

## Voorspellingsfunctie opslaan met cloudpickle

In [ ]:
# Pickle voorspellingsfunctie
print("Saving prediction function with cloudpickle...")
p = cloudpickle.dumps(predict)
with open("numerai_sparkling_water_model.pkl", "wb") as f:
    f.write(p)

print("Prediction function saved as 'numerai_sparkling_water_model.pkl'")

## Kaggle specifieke functies voor het opslaan van resultaten

In [ ]:
# Opslaan van resultaten in Kaggle output
# Dit maakt het mogelijk om de resultaten te downloaden of als dataset te gebruiken
try:
    # Maak een output directory
    !mkdir -p /kaggle/working/output
    
    # Kopieer de belangrijke bestanden
    !cp numerai_sparkling_water_model.pkl /kaggle/working/output/
    !cp {mojo_path} /kaggle/working/output/
    
    print("Model bestanden opgeslagen in Kaggle output directory")
except Exception as e:
    print(f"Fout bij opslaan in Kaggle output: {e}")

## Voordelen van Sparkling Water

In [ ]:
# Hier zou je een vergelijking kunnen maken tussen standaard H2O en Sparkling Water
print("Sparkling Water Voordelen:")
print("1. Gedistribueerde verwerking met Spark voor grote datasets")
print("2. Combinatie van Spark's data processing met H2O's machine learning algoritmes")
print("3. Betere schaalbaarheid voor complexe modellen en grote datasets")
print("4. Mogelijkheid om Spark ML Pipeline te integreren met H2O modellen")
print(f"5. Onze training duurde {training_time:.2f} seconden met Sparkling Water")

## Afsluiten van Spark en H2O

In [ ]:
# Sluit H2O cluster af
h2o.cluster().shutdown()

# Sluit Spark sessie af
spark.stop()